# Training ML algos on EPH. Predicting on CENSO.

In [1]:
# Load modules
# import time
# time.sleep(2000)

import pandas as pd
import numpy as np


# Column names
y_cols = ['CAT_OCUP', 'P47T', 'PP10E', 'PP10D', 'PP07K', 'PP07I', 'V3_M', 'PP07G4', 'CH16', 'T_VI', 
          'V12_M', 'TOT_P12', 'PP07G3', 'V5_M', 'PP07H', 'V2_M', 'PP10C', 
          'PP08D1', 'PP07J', 'CAT_INAC', 'CH07', 'CH08', 'P21', 'PP07G1', 'PP07G_59', 'PP07G2']


x_cols = ['IX_TOT', 'P02', 'P03', 'AGLOMERADO', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05', 'CONDACT']

# Sector de testeo. Codigo posta mas abajo!

### Import data

In [2]:

# T = '20162'
# # Load EPH
# # PBA_train = pd.read_csv('./PBA_train.csv').fillna(0)
# PBA_train = pd.read_csv('./PBA_train_'+T+'.csv').fillna(0)
# # PBA_train = PBA_train.sample(frac = 1).reset_index(drop = True)
# # PBA_train.head()

# # Load Census
# # X_censo = pd.read_csv('./sample_censo_table025_A_bolivar.csv').fillna(0)
# # X_censo = pd.read_csv('./sample_censo_table025_A_vlopez_rodriguez.csv').fillna(0)
# X_censo = pd.read_csv('./sample_censo_table025rand.csv', nrows=300000, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
#        'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
#        'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0).rename(columns = {'CONDACT': 'CA'})
# X_censo[x_cols].values.shape

In [3]:
# # PBA_train.groupby('P03').count()#['CONDACT']

# df_25_50_emp = PBA_train[['P03','ANO4', 'TRIMESTRE','CONDACT', 'CAT_OCUP', 'CAT_INAC']]
# sum_e = pd.get_dummies(df_25_50_emp, columns = ['CONDACT', 'CAT_OCUP', 'CAT_INAC']).groupby(['ANO4','TRIMESTRE']).sum()
# total_e = df_25_50_emp.groupby(['ANO4','TRIMESTRE']).count().iloc[:, 1]
# perce = 100*np.round(sum_e.div(total_e, 0), 4)

In [4]:
# perce

### Scale and transform some features. 
Eg we do not want any confusion on CONDACT, AGLO so we scale them up. We also want ~10 age ranges, not 100 exact age values. But pay attention that the same transformation needs to be on the EPH and the CENSUS Columns. Eventually they sould be taken back. Eg, we finally need exact age values and not transformed ones. We take care of that when we load the data for other stuff.

In [5]:
# # Escalar algunos de los features
# # Grupo edad (deciles)

# # Numbers 0 to 9 for age deciles. 
# # Estas en el decil de edad. x% de la gente menor que uno.
# # PBA_train['P03'] = np.floor(10*PBA_train[['P03']].rank(pct = 1)-0.01).astype(int)
# # X_censo['P03'] = np.floor(10*X_censo[['P03']].rank(pct = 1)-0.01).astype(int)

# PBA_train['P03'] = (abs(PBA_train[['P03']] -19) < 3).astype(int) * PBA_train[['P03']] + \
# (abs(PBA_train[['P03']] -19) >= 3).astype(int) * np.floor(10*PBA_train[['P03']].rank(pct = 1)-0.01).astype(int)

# X_censo['P03'] = (abs(X_censo[['P03']] -19) < 3).astype(int) * X_censo[['P03']] + \
# (abs(X_censo[['P03']] -19) >= 3).astype(int) * np.floor(10*X_censo[['P03']].rank(pct = 1)-0.01).astype(int)

# # train['CH07'] = train['CONDACT']

# # Superweight CONDACT
# PBA_train['CONDACT'] = 100*PBA_train['CONDACT']
# X_censo['CONDACT'] = 100*X_censo['CONDACT']

# # Superweight CONDACT
# PBA_train['AGLOMERADO'] = 10*PBA_train['AGLOMERADO']
# X_censo['AGLOMERADO'] = 10*X_censo['AGLOMERADO']


## Three step fitting and predicting on Censo
*Classification.* Use data pooled from all TRIMESTRE. Income does not play a role. Knowing this info better we can predict income in the third step.
#### Step 1: Classification RF for 'estado civil' and 'condicion  de actividad', eg Jubilado, estudiante, ama de casa, ninos, trabajador.
#### Step 2: Classification RF for questions on employment quality, i.e. obra social, vacaciones pagas, cobra en blanco.

*Regression:* Each ANO4 TRIMESTRE has wage levels. Especially so in Argentina because of high inflation.
Here the training dataset is different, it uses the ANO4 and TRIMESTRE columns and includes the income variables.
#### Step 3: Regression RF for income variables.

### In each step we fit on EPH and predict on CENSO. So at the end we have the completed CENSO dataset ready to save.

In [6]:
# from sklearn.model_selection import train_test_split

# import numpy as np


# ### STEP 1
# predecir = ['CONDACT', 'CAT_OCUP', 'CAT_INAC', 'CH07']

# X = PBA_train[x_cols]
# y = PBA_train[predecir].loc[X.index]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
# # Rewrite Census data as 'test' set.

# from sklearn import tree
# from sklearn.ensemble import RandomForestClassifier

# X = X_train
# y = y_train

# # clf = tree.DecisionTreeClassifier()
# clf = RandomForestClassifier(n_estimators=50)
# clf = clf.fit(X.values, y.values)

# y_out1 = clf.predict(X_censo[x_cols].values)
# y_censo_fit1 = pd.DataFrame(y_out1, index = X_censo.index, columns=predecir)
# Xy1_censo = pd.concat([X_censo, y_censo_fit1], axis = 1)

# ### STEP 2

# x_cols2 = x_cols+predecir
# # La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
# predecir2 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
# y_cols2 = predecir2
# X = PBA_train[x_cols2]
# y = PBA_train[y_cols2].loc[X.index]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
# X = X_train; y = y_train

# # clf2 = tree.DecisionTreeClassifier()
# # clf2 = clf2.fit(X.values, y.values)
# clf2 = RandomForestClassifier(n_estimators=50).fit(X.values, y.values)

# y_out2 = clf2.predict(Xy1_censo[x_cols2].values)
# y_censo_fit2 = pd.DataFrame(y_out2, index = X_censo.index, columns=predecir2)
# Xy2_censo = pd.concat([Xy1_censo, y_censo_fit2], axis = 1)

# # ### STEP 3 (Regression)

# # x_cols3 = x_cols2 + predecir2
# # # Columnas de ingresos. Necesitan una regresion...
# # predecir3 = ['P47T', 'V3_M', 'T_VI', 'V12_M', 'TOT_P12', 'V5_M','V2_M', 'PP08D1', 'P21']
# # y_cols3 = predecir3

# # X = EPH[x_cols3].sample(frac = 1) #PBA_train_reg
# # y = EPH[y_cols3].loc[X.index] #PBA_train_reg

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# # X = X_train; y = y_train

# # clf3 = tree.DecisionTreeRegressor()
# # clf3 = clf3.fit(X.values, y.values)

# # y_out = clf3.predict(X_test.values)
# # y_test_fit = pd.DataFrame(y_out, index = X_test.index, columns=predecir3)
# # both = pd.concat([X_test, y_test[predecir3], y_test_fit], axis = 1)

# # y_out3 = clf3.predict(Xy2_censo[x_cols3].values)
# # y_censo_fit3 = pd.DataFrame(y_out3, index = X_censo.index, columns=predecir3)
# # Xy3_censo = pd.concat([Xy2_censo, y_censo_fit3], axis = 1)

# # Xy3_censo = round(Xy3_censo)

### Save outcoming Censo dataset

In [7]:
# # Xy2_censo.to_csv('RFC_25rand.csv', index = False)

# Xy2_censo.to_csv('RFC_25rand_.csv', index = False)

In [8]:
# pd.options.display.max_columns = 999
# Xy2_censo.head()
# # Xy3_censo.iloc[:, -4].value_counts().sort_index()
# #df.sum()/25/1e6

In [9]:
# Xy3_censo.drop('Unnamed: 0', axis = 1).to_csv('censo_filled_vloprodr.csv', index = False)

# Ahora si!
## Train on Quarterly data

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
# import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [11]:

# T = '20162'
# # Load EPH
# # PBA_train = pd.read_csv('./PBA_train.csv').fillna(0)
# PBA_train = pd.read_csv('./PBA_train_'+T+'.csv').fillna(0)
# # PBA_train = PBA_train.sample(frac = 1).reset_index(drop = True)
# # PBA_train.head()

# # Load Census
# # X_censo = pd.read_csv('./sample_censo_table025_A_bolivar.csv').fillna(0)
# # X_censo = pd.read_csv('./sample_censo_table025_A_vlopez_rodriguez.csv').fillna(0)
# X_censo = pd.read_csv('./sample_censo_table025rand.csv', nrows=300000, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
#        'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
#        'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0).rename(columns = {'CONDACT': 'CA'})
# X_censo[x_cols].values.shape

In [12]:
# Load Census
file = './sample_censo_table_f.1BA.csv'

X_censo = pd.read_csv(file, usecols = ['DPTO','RADIO_REF_ID','PERSONA_REF_ID','IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05']).fillna(0)
X_censo[x_cols].values.shape

X_censo['P03'] = (abs(X_censo[['P03']] -19) < 3).astype(int) * X_censo[['P03']] + \
(abs(X_censo[['P03']] -19) >= 3).astype(int) * np.floor(10*X_censo[['P03']].rank(pct = 1)-0.01).astype(int)
X_censo['CONDACT'] = 100*X_censo['CONDACT']
X_censo['AGLOMERADO'] = 10*X_censo['AGLOMERADO']

In [13]:
X_censo.shape

(1852778, 27)

In [14]:
# Load EPH

PBA_train = pd.read_csv('./PBA_train.csv').fillna(0)
PBA_train['P03'] = (abs(PBA_train[['P03']] -19) < 3).astype(int) * PBA_train[['P03']] + \
(abs(PBA_train[['P03']] -19) >= 3).astype(int) * np.floor(10*PBA_train[['P03']].rank(pct = 1)-0.01).astype(int)
PBA_train['CONDACT'] = 100*PBA_train['CONDACT']
PBA_train['AGLOMERADO'] = 10*PBA_train['AGLOMERADO']

# PBA_train = PBA_train.loc[PBA_train.AGLOMERADO.isin(X_censo['AGLOMERADO'].unique())]


In [16]:

### STEP 1
x_cols1 = x_cols

#     predecir = ['CAT_OCUP', 'CAT_INAC', 'CH07']
predecir1 = ['CAT_OCUP', 'CAT_INAC', 'CH07']
y_cols1 = predecir1

X = PBA_train[x_cols1]
y = PBA_train[y_cols1].loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
# Rewrite Census data as 'test' set.

X = X_train; y = y_train

# clf = tree.DecisionTreeClassifier()
clf1 = RandomForestClassifier(n_estimators=50)
clf1 = clf1.fit(X.values, y.values)

y_out1 = clf1.predict(X_censo[x_cols1].values)
y_censo_fit1 = pd.DataFrame(y_out1, index = X_censo.index, columns=predecir1)
Xy1_censo = pd.concat([X_censo, y_censo_fit1], axis = 1)

### STEP 2

x_cols2 = x_cols1+predecir1
# La seccion PP07G pregunta si el trabajo es en blanco y que beneficios tiene. Puede ayudar a la regresion para ingresos.
predecir2 = ['PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K']
y_cols2 = predecir2
X = PBA_train[x_cols2]
y = PBA_train[y_cols2].loc[X.index]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
X = X_train; y = y_train

# clf2 = tree.DecisionTreeClassifier()
# clf2 = clf2.fit(X.values, y.values)
clf2 = RandomForestClassifier(n_estimators=50).fit(X.values, y.values)

y_out2 = clf2.predict(Xy1_censo[x_cols2].values)
y_censo_fit2 = pd.DataFrame(y_out2, index = X_censo.index, columns=predecir2)
Xy2_censo = pd.concat([Xy1_censo, y_censo_fit2], axis = 1)


In [17]:
# # save RFC_25rand_20162
# Xy2_censo.to_csv('RFC_.1BA.csv', index = False)

In [18]:
for T in ['20162', '20163', '20164', '20171', '20172',  '20173',  '20174',  '20181', '20182', '20183']:
    print(T)
    ### STEP 3 (Regression)
    PBA_train = pd.read_csv('./PBA_train_'+T+'.csv').fillna(0)

    x_cols3 = x_cols2 + predecir2
    # Columnas de ingresos. Necesitan una regresion...
    predecir3 = ['P47T', 'V3_M', 'T_VI', 'V12_M', 'TOT_P12', 'V5_M','V2_M', 'PP08D1', 'P21']
    y_cols3 = predecir3

    X = PBA_train[x_cols3].sample(frac = 1) #PBA_train_reg
    y = PBA_train[y_cols3].loc[X.index] #PBA_train_reg

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X = X_train; y = y_train

    clf3 = RandomForestRegressor(n_estimators=50)
    clf3 = clf3.fit(X.values, y.values)

    y_out3 = clf3.predict(Xy2_censo[x_cols3].values)
    y_censo_fit3 = pd.DataFrame(y_out3, index = X_censo.index, columns=predecir3)
    Xy3_censo = pd.concat([Xy2_censo, y_censo_fit3], axis = 1)
    Xy3_censo['ANO4'] = T[:-1]; Xy3_censo['TRIMESTRE'] = T[-1]

    
    # save
#     Xy3_censo.to_csv('RFReg_.01AR'+T+'.csv', index = False)
    Xy3_censo.to_csv('RFReg_.1BA'+T+'.csv', index = False)

20162
20163
20164
20171
20172
20173
20174
20181
20182
20183


In [ ]:
Xy3_censo.head()